In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# 分析地址数据的格式：
# col   含义
# 0     ID
# 1     运单ID
# 2     类型
# 3     省
# 4     市
# 5     区
# 6     gpsX
# 7     gpsY
# 8     主节点
# 9     顺序

In [3]:
address_A_path = r'Bdata/address_B_easy.txt'
waybill_A_path = r'Bdata/waybill_B.txt'

address_A = pd.read_csv(address_A_path,header=None)
waybill_A = pd.read_csv(waybill_A_path,header=None)
# print(address_A)
# print(waybill_A)


loadCity_A_path = r'Bdata/loadCity_B_pro.txt'
UnloadCity_A_path = r'Bdata/UnloadCity_B_pro.txt'

loadCity_A = pd.read_csv(loadCity_A_path,header=None)
UnloadCity_A = pd.read_csv(UnloadCity_A_path,header=None)

caseNumber1 = loadCity_A.index.stop
caseNumber2 = UnloadCity_A.index.stop

result_out=[]


for p in range(0,caseNumber1):
    for q in tqdm(range(0,caseNumber2)):
#         print(loadCity_A.loc[p].values[0],loadCity_A.loc[p].values[1],
#               UnloadCity_A.loc[q].values[0],UnloadCity_A.loc[q].values[1])
        
#         Pos_A_Pro=r"陕西"
#         Pos_A_City=r"咸阳"
#         Pos_B_Pro=r"陕西"
#         Pos_B_City=r"西安"
        Pos_A_Pro =loadCity_A.loc[p].values[0]
#         Pos_A_City=loadCity_A.loc[p].values[1]
        Pos_B_Pro= UnloadCity_A.loc[q].values[0]
#         Pos_B_City=UnloadCity_A.loc[q].values[1]

        currentId=''
        caseNumber = address_A.index.stop
        i = 0
        count = 1
        resultIds=[]
        while i<caseNumber:

            address= address_A.loc[i].values
            addressId=address[1]
            addressPro=address[3]
            addressCity=address[4]
            #print(addressId,addressPro,addressCity)

            i=i+1
            if addressId==currentId: #如果不是新的ID
                continue
            else: #如果是新的运单ID
                currentId=addressId
#                 if addressPro==Pos_A_Pro and addressCity==Pos_A_City:
                if addressPro==Pos_A_Pro :
                    #如果是目标提货点Pos_A,直接搜索下一条数据
                    address= address_A.loc[i].values
                    addressId=address[1]
                    addressPro=address[3]
                    addressCity=address[4]
#                     if addressPro==Pos_B_Pro and addressCity==Pos_B_City:
                    if addressPro==Pos_B_Pro :
                        #如果是目标卸货点Pos_B
                        #在运单数据中，记录该订单

                        resultIds.append(addressId)
                        #print(count,addressId)
                        #count=count+1

                else:#如果不是目标提货点Pos_A
                    continue

            i=i+1


        #进行核心计算
        # address= address_A.loc[i].values
        caseNumber = waybill_A.index.stop
        i = 0
        vols=[0]*19
        tons=[0]*19

        while i<caseNumber:
            waybill=waybill_A.loc[i].values

            times = waybill[2].split('-') #拿到运单开始时间分段
            year=times[0]
            month=times[1]
            currentId=waybill[0]
            vol=waybill[4]
            ton=waybill[5]
        #     print(currentId,year,month,vol,ton)

            if currentId in resultIds:
                Id=(int(year)-2018)*12+int(month)-1
                vols[Id]=vols[Id]+vol
                tons[Id]=tons[Id]+ton

            i=i+1

        # print(len(vols),vols)
        # print(len(tons),tons)
        
        result_out.append([Pos_A_Pro,
#                           Pos_A_City,
                          Pos_B_Pro,
#                           Pos_B_City,
                          vols,
                          tons]
                          )
        
   


100%|█████████████████████████████████████████████| 31/31 [07:32<00:00, 14.61s/it]


In [4]:
df=pd.DataFrame(result_out)
print(df)

       0    1                                                  2  \
0     河北   安徽  [1480.0, 1200.0, 1283.0, 0, 290.0, 0, 110.0, 0...   
1     河北   山东  [40.0, 360.0, 60.0, 120.0, 100.0, 460.0, 100.0...   
2     河北   浙江  [50.0, 0, 0, 0, 100.0, 0, 0, 0, 700.0, 0, 0, 2...   
3     河北   江苏  [0, 180.0, 0, 0, 0, 0, 100.0, 0, 400.0, 0, 0, ...   
4     河北   湖北  [0, 0, 0, 0, 100.0, 300.0, 0, 300.0, 100.0, 0,...   
..   ...  ...                                                ...   
863  内蒙古   甘肃  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
864  内蒙古   北京  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
865  内蒙古   西藏  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
866  内蒙古  黑龙江  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
867  内蒙古   青海  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                                     3  
0    [360.0, 273.0, 417.0, 0, 78.3, 0, 35.0, 0, 30....  
1    [19.0, 88.0, 10.0, 40.0, 30.0, 140.0, 30.0, 30...  
2    [9.6, 0

In [5]:
outputpath=r'Bdata/result.csv'
df.to_csv(outputpath,index=False,header=False,encoding='utf_8_sig')